In [471]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

In [472]:
data= pd.read_csv('C:/Users/HP/Desktop/mon_stage_senegal/data/Etudiants_2001_2024/Liste_globale_des_etudiants_tous_les_champs20012002.csv' , encoding='latin1' ,sep=';')


In [473]:
data.columns

Index(['NUMERO', 'NUMERO_TABLE', 'INE', 'NUMERO_IDENTITIFICATION', 'NOM',
       'PRENOM', 'DATE_DE_NAISSANCE', 'LIEU_DE_NAISSANCE',
       'MAIL_INSTITUTIONNEL', 'SEXE', 'MENTION_BACC', 'ANNEE_BACC',
       'PAYS_DE_NAISSANCE', 'NATIONALITE', 'SERIE_BACC', 'REGION_DE_NAISSANCE',
       'ETABLISSMENT_CODE', 'ETABLISSEMENT', 'NIVEAU_SECTION',
       'SIGLE_NIVEAU_SECTION', 'COHORTE', 'ETAT_INSCRIPTION',
       'ANNEE_INSCRIPTION', 'ANNEE_UNIVERSITAIRE', 'TYPE_FORMATION',
       'CODE_NIVEAU', 'NIVEAU', 'DATE_INSCRIPTION', 'NIVEAU LMD',
       'NIVEAU LMD ET NON LMD', 'SYSTEME', 'DEPARTEMENT FORMATION'],
      dtype='object')

In [474]:
#fonction  les  informations du dataframe
def afficher_info(df):
    print("#==============Informations sur les anomalie du fichier================ :")
    print(f"il y'a {df.shape[0]} lignes et {df.shape[1]} colonnes.")

afficher_info(data)

#==============Informations sur les anomalie du fichier================ :
il y'a 29094 lignes et 32 colonnes.


In [475]:
#======= Anomalie1 : les personnes avec un systeme lmd inscrit avant 2010 =========
def anomalie1(df):
    anomalie1_df = df[(df['ANNEE_INSCRIPTION'] < 2011) & (df['SYSTEME'] == 'LMD')]
    if len(anomalie1_df) == 0:
        print("Il n'y a pas de personnes avec un système LMD inscrit avant 2011.")
    else:
     print(f"il y'a {len(anomalie1_df)} personnes avec un systeme lmd inscrit inscrit avant 2011  .")
    
anomalie1(data) 

il y'a 19235 personnes avec un systeme lmd inscrit inscrit avant 2011  .


In [476]:
#=========== Anomalie2 :  les valeurs manquantes dans les colonnes  ==========
def anomalie2(df):
    anomalie2_df = df.isnull().sum()
    anomalie2_df = anomalie2_df[anomalie2_df > 0]
    print(" le nombre de valeur manquante pour chaque colonne   :")
    print(anomalie2_df )
anomalie2(data)    

 le nombre de valeur manquante pour chaque colonne   :
NUMERO_TABLE           29094
INE                    27181
LIEU_DE_NAISSANCE        104
MAIL_INSTITUTIONNEL    25650
MENTION_BACC             334
ANNEE_BACC               279
SERIE_BACC               435
REGION_DE_NAISSANCE    10534
COHORTE                26738
NIVEAU LMD              9859
dtype: int64


ici on vas reperer les année de naisance anormales 
tous les personnes ayant passant le bac  avant  14 ans ou apres 40 ans sont consideres comme anormales 
les personnes qui ont passes leur bac apres 30ans sont pas impossible mais tres rare 

In [ ]:
#======anomalie3 :  les annéee de naissance pas bon ===============


def anomalie3(data):
    # Convertir la colonne 'DATE_DE_NAISSANCE' en datetime
    data['DATE_DE_NAISSANCE'] = pd.to_datetime(data['DATE_DE_NAISSANCE'], errors='coerce')
    # Extraire l'année de naissance
    data['annee_naissance'] = data['DATE_DE_NAISSANCE'].dt.year
    # Nettoyer et extraire uniquement les années de bac à 4 chiffres
    data['ANNEE_BACC'] = pd.to_numeric(
        data['ANNEE_BACC'].astype(str).str.extract(r'(\d{4})')[0], 
        errors='coerce'
    )
   # Calculer l'âge au bac uniquement pour les lignes valides
    data['age_bac'] = data['ANNEE_BACC'] - data['annee_naissance']
   # Rechercher les personnes ayant passé le bac avant 14 ans
    personnes_bac_avant_14_ans = data[data['age_bac'] < 14]
    personnes_bac_apres_40ans = data[data['age_bac'] > 40]
    personnes_bac_apres_30ans = data[data['age_bac'] > 30]
    print(f"Il y a {personnes_bac_avant_14_ans.shape[0]} personnes qui ont passé le bac avant 14 ans. voici leur année de naissance et l'année de bac : \t")
    print(personnes_bac_avant_14_ans[['DATE_DE_NAISSANCE', 'ANNEE_BACC']])
    print(f"Il y a {personnes_bac_apres_40ans.shape[0]} personnes qui ont passé le bac avant 40 ans. voici leur année de naissance et l'année de bac : \t")
    print(personnes_bac_apres_40ans[['DATE_DE_NAISSANCE', 'ANNEE_BACC']])
    print(f"Il y a {personnes_bac_apres_30ans.shape[0]} personnes qui ont passé le bac avant 30 ans.! pas impossible mais rare")

    
anomalie3(data)     

Il y a 17 personnes qui ont passé le bac avant 14 ans. voici leur année de naissance et l'année de bac : 	
      DATE_DE_NAISSANCE  ANNEE_BACC
205          1969-02-10      1981.0
2408         1978-12-06      1990.0
3207         1980-02-01      1993.0
3292         1977-07-03      1977.0
6309         1975-12-01      1979.0
6448         1972-05-06      1984.0
7134         1974-11-03      1974.0
9273         1976-04-10      1988.0
12114        1992-06-09      2001.0
16097        2098-09-12      2001.0
17347        1976-07-03      1977.0
17857        1973-05-02      1973.0
24405        1977-11-11      1977.0
26663        1978-11-01      1988.0
28097        1974-10-09      1974.0
29020        1979-03-03      1991.0
29068        1972-05-07      1972.0
Il y a 10 personnes qui ont passé le bac avant 40 ans. voici leur année de naissance et l'année de bac : 	
      DATE_DE_NAISSANCE  ANNEE_BACC
4682         1954-01-09      1997.0
5752         1980-10-07      2201.0
10491        1956-07-03      1

on veut reperer les personnes qui font deux ou tois formations differentes 

In [478]:
def anomalie4(data):
    # Compter les doublons exacts (lignes identiques)
    total = data.duplicated(subset='NUMERO').sum()
    
    # Filtrer les étudiants apparaissant plusieurs fois avec le même numéro
    d = data[data.duplicated(subset=['NUMERO'], keep=False)]

    # Compter le nombre d'établissements différents pour chaque étudiant
    totale1 = d.groupby('NUMERO')['ETABLISSMENT_CODE'].nunique()

    # Compter combien d'étudiants ont étudié dans plusieurs établissements
    nb_personnes_2_formations = (totale1 == 2).sum()
    nb_personnes_3_formations = (totale1 == 3).sum()
    nb_personnes_4_formations = (totale1 == 4).sum()
    nb_personnes_plus_4 = (totale1 > 4).sum()

    print(f"Il y a {total} doublons exacts dans le DataFrame.")
    print(f"Il y a {nb_personnes_2_formations} personnes qui ont étudié dans 2 établissements différents.")
    print(f"Il y a {nb_personnes_3_formations} personnes qui ont étudié dans 3 établissements différents.")
    print(f"Il y a {nb_personnes_4_formations} personnes qui ont étudié dans 4 établissements différents.")
    print(f"Il y a {nb_personnes_plus_4} personnes qui ont étudié dans plus de 4 établissements différents.")
    
    
anomalie4(data)    


Il y a 243 doublons exacts dans le DataFrame.
Il y a 122 personnes qui ont étudié dans 2 établissements différents.
Il y a 0 personnes qui ont étudié dans 3 établissements différents.
Il y a 0 personnes qui ont étudié dans 4 établissements différents.
Il y a 0 personnes qui ont étudié dans plus de 4 établissements différents.


In [479]:
#afficher les mentions de bac valides 
data['MENTION_BACC'].unique()

array(['PA', 'AB', 'BI', nan, 'TB', 'ba', 'p', 'TH', 'HO', 'DI', 'DA',
       '50', 'ta', 'A', 'Ka', 'da', 'P', '0', '15'], dtype=object)

on remarque des mentions de bac pas normales comme '50' , '15' etc........

In [480]:
#========== anomalie5 : les mentions au bac pas normales  ===========
def anomalie5(data):
    #les mentions au bac pas normales
    compteur = 0 
    mentions_normales  = ['AB', 'B', 'TB', 'PA']
    #compter le nombre de mentions anormales
    for i in data['MENTION_BACC']:
        if i not in mentions_normales:
            compteur+=1 
        
    print(f"il y'a {compteur} mentions anormales au bac")
    #afficher les mentions anormales
    mentions_anormales = data[~data['MENTION_BACC'].isin(mentions_normales)]['MENTION_BACC'].unique()
    print("les mentions anormales sont : ", mentions_anormales)
    
anomalie5(data)   

il y'a 798 mentions anormales au bac
les mentions anormales sont :  ['BI' nan 'ba' 'p' 'TH' 'HO' 'DI' 'DA' '50' 'ta' 'A' 'Ka' 'da' 'P' '0'
 '15']


les personnes qui ont basse apres le bac apartir de 1998 auront comme serie de bac [L1B,S1A,S1,L1A,S3,S4,T1,STEG,SEA,F6,S5,S2,STIDD,LA,T2,L-AR,L2,L'1]
les personnes qui ont passe le bac avant 1998 auront comme seri series_bac_avant_1998 = ["A1", "A2", "A3", "A4", "C", "D", "E",  "F1", "F2", "F3", "F4" ,"F5", "F6", "F7",  "G1", "G2", "G3",  "T1", "T2",  "L'", "S'"  ]

In [481]:
data['SERIE_BACC'].unique()

array(['C', 'S2', 'D', 'B', 'G2', 'A3', 'A', "L'1", 'L2', nan, 'S1', 'G',
       'A4', 'E', 'A2', 'S3', 'S', 'T1', 'T2', 'a3', "L'", 'F7', 'F2'],
      dtype=object)

In [482]:
#==========anomalie6 : les series de bac pas valides =======
def anomalie6(data):
    #les series de bac pas valides
    compteur = 0 
    compteur1 = 0
    series_normales_avant_98 = [ "A1", "A2", "A3", "A4",  "C", "D", "E","F1", "F2", "F3", "F4", "F5", "F6", "F7", "G1", "G2", "G3",  "T1", "T2",  "L'", "S'"  ]  
    series_normales_apres_98 = ["L1B", "S1A", "S1", "L1A", "S3", "S4", "T1", "STEG", "SEA", "F6","S5", "S2", "STIDD", "LA", "T2", "L-AR", "L2", "L'1"]         
     #convertir la colonne 'ANNEE_BACC' en entier
    data['ANNEE_BACC'] = pd.to_numeric(data['ANNEE_BACC'], errors='coerce')
    #compter le nombre de series anormales
    for _, i in data.iterrows():
        if i['SERIE_BACC'] not in series_normales_avant_98 and i['ANNEE_BACC'] < 1998:
            compteur+=1 
        if i['SERIE_BACC'] not in series_normales_apres_98 and i['ANNEE_BACC'] >= 1998:
            compteur1+=1    
        
    print(f"il y'a {compteur} qui ont passe leur bac avant  1998 qui une serie de bac pas normale")
    #afficher les series anormales
    series_anormales = data[~data['SERIE_BACC'].isin(series_normales_avant_98) & (data['ANNEE_BACC'] < 1998)]['SERIE_BACC'].unique()
    print("ces series sont : ", series_anormales)
    print(f"il y'a {compteur1} qui ont passe leur bac apres  1998 qui une serie de bac pas normale")
    #afficher les series anormales
    series_anormales1 = data[~data['SERIE_BACC'].isin(series_normales_apres_98) & (data['ANNEE_BACC'] >= 1998)]['SERIE_BACC'].unique()
    print("ces series sont : ", series_anormales1)
    
     
anomalie6(data)    

il y'a 1150 qui ont passe leur bac avant  1998 qui une serie de bac pas normale
ces series sont :  ['B' 'A' 'S2' nan "L'1" 'S' 'L2' 'S1' 'S3' 'G' 'a3']
il y'a 2693 qui ont passe leur bac apres  1998 qui une serie de bac pas normale
ces series sont :  ['G2' 'A3' nan 'A' 'C' 'D' 'G' 'B' 'S' 'E' 'A2' 'A4' 'a3' "L'" 'F7']


In [483]:
#==========anomalie6 : les series de bac pas valides =======
def anomalie6(data):
    #les series de bac pas valides  
    compteur = 0 
    compteur1 = 0
    series_normales_avant_98 = [ "A1", "A2", "A3", "A4",  "C", "D", "E","F1", "F2", "F3", "F4", "F5", "F6", "F7", "G1", "G2", "G3",  "T1", "T2",  "L'", "S'"  ]  
    series_normales_apres_98 = ["L1B", "S1A", "S1", "L1A", "S3", "S4", "T1", "STEG", "SEA", "F6","S5", "S2", "STIDD", "LA", "T2", "L-AR", "L2", "L'1"]         
     #convertir la colonne 'ANNEE_BACC' en entier
    data['ANNEE_BACC'] = pd.to_numeric(data['ANNEE_BACC'], errors='coerce')
    #compter le nombre de series anormales
    for _, i in data.iterrows():
        if i['SERIE_BACC'] not in series_normales_avant_98 and i['ANNEE_BACC'] < 1998:
            compteur+=1 
        if i['SERIE_BACC'] not in series_normales_apres_98 and i['ANNEE_BACC'] >= 1998:
            compteur1+=1    
        
    print(f"il y'a {compteur} qui ont passe leur bac avant  1998 qui une serie de bac pas normale")
    #afficher les series anormales
    series_anormales = data[~data['SERIE_BACC'].isin(series_normales_avant_98) & (data['ANNEE_BACC'] < 1998)]['SERIE_BACC'].unique()
    print("ces series sont : ", series_anormales)
    print(f"il y'a {compteur1} qui ont passe leur bac apres  1998 qui une serie de bac pas normale")
    #afficher les series anormales
    series_anormales1 = data[~data['SERIE_BACC'].isin(series_normales_apres_98) & (data['ANNEE_BACC'] >= 1998)]['SERIE_BACC'].unique()
    print("ces series sont : ", series_anormales1)
    
     
anomalie6(data)    

il y'a 1150 qui ont passe leur bac avant  1998 qui une serie de bac pas normale
ces series sont :  ['B' 'A' 'S2' nan "L'1" 'S' 'L2' 'S1' 'S3' 'G' 'a3']
il y'a 2693 qui ont passe leur bac apres  1998 qui une serie de bac pas normale
ces series sont :  ['G2' 'A3' nan 'A' 'C' 'D' 'G' 'B' 'S' 'E' 'A2' 'A4' 'a3' "L'" 'F7']


In [484]:
#================= afficher toutes les anomalies ===================
def afficher_tous_les_anomalies(data):
        afficher_info(data)
        anomalie1(data)
        anomalie2(data)
        anomalie3(data)
        anomalie4(data)
        anomalie5(data)
        anomalie6(data)
        



In [485]:
#exemple d'aaplication de la fonction
afficher_tous_les_anomalies(data)

#==============Informations sur les anomalie du fichier================ :
il y'a 29094 lignes et 34 colonnes.
il y'a 19235 personnes avec un systeme lmd inscrit inscrit avant 2011  .
 le nombre de valeur manquante pour chaque colonne   :
NUMERO_TABLE           29094
INE                    27181
DATE_DE_NAISSANCE      16099
LIEU_DE_NAISSANCE        104
MAIL_INSTITUTIONNEL    25650
MENTION_BACC             334
ANNEE_BACC               319
SERIE_BACC               435
REGION_DE_NAISSANCE    10534
COHORTE                26738
NIVEAU LMD              9859
annee_naissance        16099
age_bac                16232
dtype: int64
Il y a 17 personnes qui ont passé le bac avant 14 ans. voici leur année de naissance et l'année de bac : 	
      DATE_DE_NAISSANCE  ANNEE_BACC
205          1969-02-10      1981.0
2408         1978-12-06      1990.0
3207         1980-02-01      1993.0
3292         1977-07-03      1977.0
6309         1975-12-01      1979.0
6448         1972-05-06      1984.0
7134         

In [486]:
import os
import os
import io
import contextlib
import pandas as pd

def exporter_anomalies(fichier_entree ):
    # Lire le fichier (Excel ou CSV automatiquement détecté)
    extension = os.path.splitext(fichier_entree)[1].lower()
    if extension == '.csv':
        data = pd.read_csv(fichier_entree , encoding='latin1', sep=';')
    elif extension in ['.xls', '.xlsx']:
        data = pd.read_excel(fichier_entree)
    else:
        raise ValueError("Format de fichier non supporté. Utilisez .csv, .xls ou .xlsx")

    # Construire le nom du fichier de sortie
    nom_base = os.path.splitext(os.path.basename(fichier_entree))[0]
    nom_sortie = f"anomalies_{nom_base}.txt"

    # Liste des fonctions d'anomalies
    fonctions_anomalies = [anomalie1, anomalie2, anomalie3, anomalie4, anomalie5 , anomalie6]

    # Rediriger la sortie print vers un fichier texte
    with open(nom_sortie, 'w', encoding='utf-8') as f:
        for i, fonction in enumerate(fonctions_anomalies, 1):
            f.write(f"===== Anomalie {i} =====\n")
            buffer = io.StringIO()
            with contextlib.redirect_stdout(buffer):
                fonction(data)
            f.write(buffer.getvalue())
            f.write("\n\n")

    

    


In [487]:
exporter_anomalies('C:/Users/HP/Desktop/mon_stage_senegal/data/Etudiants_2001_2024/Liste_globale_des_etudiants_tous_les_champs20012002.csv')